In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator, TabularDataset

import numpy as np

ModuleNotFoundError: No module named 'sentencepiece'

In [0]:
import json
train_hi = open('/content/drive/My Drive/IIIT Classes/enghin/train.hi','r').read().split('\n')
train_en = open('/content/drive/My Drive/IIIT Classes/enghin/train.en','r').read().split('\n')
test_hi = open('/content/drive/My Drive/IIIT Classes/enghin/test.hi','r').read().split('\n')
test_en = open('/content/drive/My Drive/IIIT Classes/enghin/test.en','r').read().split('\n')
dev_en = open('/content/drive/My Drive/IIIT Classes/enghin/dev.en','r').read().split('\n')
dev_hi = open('/content/drive/My Drive/IIIT Classes/enghin/dev.hi','r').read().split('\n')

f = open('/content/drive/My Drive/IIIT Classes/enghin(1)/dev.json','a')
for i in range(len(dev_hi)):
    sentences={'src':[],'trg':[]}
    for word_en in dev_en[i].split(' '):
        sentences['src'].append(word_en.lower())
    for word_hi in dev_hi[i].split(' '):
        sentences['trg'].append(word_hi)
    json.dump(sentences,f)
    f.write('\n')
f.close()

f = open('/content/drive/My Drive/IIIT Classes/enghin(1)/train.json','a')
for i in range(len(train_hi)):
    sentences={'src':[],'trg':[]}
    for word_en in train_en[i].split(' '):
        sentences['src'].append(word_en.lower())
    for word_hi in train_hi[i].split(' '):
        sentences['trg'].append(word_hi)
    json.dump(sentences,f)
    f.write('\n')
f.close()

f = open('/content/drive/My Drive/IIIT Classes/enghin(1)/test.json','a')
for i in range(len(test_hi)):
    sentences={'src':[],'trg':[]}
    for word_en in test_en[i].split(' '):
        sentences['src'].append(word_en.lower())
    for word_hi in test_hi[i].split(' '):
        sentences['trg'].append(word_hi)
    json.dump(sentences,f)
    f.write('\n')
f.close()


In [54]:
def tokenize(text):
    return text

SRC = Field(tokenize = tokenize, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

dev_data = TabularDataset(path='dev.json',format='JSON',fields={"src":('src',SRC),"trg":('trg',TRG)})
train_data = TabularDataset(path='train.json',format='JSON',fields={"src":('src',SRC),"trg":('trg',TRG)})
test_data = TabularDataset(path='test.json',format='JSON',fields={"src":('src',SRC),"trg":('trg',TRG)})

SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)


{'src': ['fresh', 'breath', 'and', 'shining', 'teeth', 'enhance', 'your', 'personality', '.'], 'trg': ['ताजा', 'साँसें', 'और', 'चमचमाते', 'दाँत', 'आपके', 'व्यक्तित्व', 'को', 'निखारते', 'हैं', '।']}


In [94]:
print(f"Unique tokens in source (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (hi) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (en) vocabulary: 17099
Unique tokens in target (hi) vocabulary: 21898


In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, dev_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)


In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [0]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [0]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [86]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(17099, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(21898, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=21898, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [91]:
import time
import random
import math

optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 5m 18s
	Train Loss: 6.130 | Train PPL: 459.597
	 Val. Loss: 6.591 |  Val. PPL: 728.642
Epoch: 02 | Time: 5m 19s
	Train Loss: 5.822 | Train PPL: 337.767
	 Val. Loss: 6.306 |  Val. PPL: 548.106
Epoch: 03 | Time: 5m 23s
	Train Loss: 5.568 | Train PPL: 261.799
	 Val. Loss: 6.244 |  Val. PPL: 514.881
Epoch: 04 | Time: 5m 20s
	Train Loss: 5.414 | Train PPL: 224.628
	 Val. Loss: 6.156 |  Val. PPL: 471.443
Epoch: 05 | Time: 5m 17s
	Train Loss: 5.256 | Train PPL: 191.783
	 Val. Loss: 6.145 |  Val. PPL: 466.536
Epoch: 06 | Time: 5m 20s
	Train Loss: 5.112 | Train PPL: 166.014
	 Val. Loss: 6.054 |  Val. PPL: 425.679
Epoch: 07 | Time: 5m 20s
	Train Loss: 4.974 | Train PPL: 144.615
	 Val. Loss: 5.960 |  Val. PPL: 387.736
Epoch: 08 | Time: 5m 18s
	Train Loss: 4.834 | Train PPL: 125.745
	 Val. Loss: 5.900 |  Val. PPL: 365.207
Epoch: 09 | Time: 5m 16s
	Train Loss: 4.676 | Train PPL: 107.314
	 Val. Loss: 5.888 |  Val. PPL: 360.714
Epoch: 10 | Time: 5m 15s
	Train Loss: 4.550 | Train PPL

In [92]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.805 | Test PPL: 331.911 |
